In [1]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import MinMaxScaler

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = 'iframe'

import ta
from ta import add_all_ta_features
from ta.utils import dropna

### 1. Read and visualize dataset

In [2]:
datasets = ['ali', 'copper', 'lead', 'nickel', 'zinc']
dataset_idx = 0

In [3]:
dataset_file = "./Data/{}.csv".format(datasets[dataset_idx])
df = pd.read_csv(dataset_file, index_col=0)
df['date'] = pd.to_datetime(df['date'])

df['month'] = df['date'].dt.month.astype(int)
df['day_of_month'] = df['date'].dt.day.astype(int)

# day_of_week=0 corresponds to Monday
df['day_of_week'] = df['date'].dt.dayofweek.astype(int)
# df['hour_of_day'] = df['date'].dt.hour.astype(int)

selected_columns = ['date', 'day_of_week', 'PX_HIGH', 'PX_LOW', 'PX_OPEN', 'PX_LAST', 'PX_VOLUME']
df = df[selected_columns]

df[df.isna().any(axis=1)]

,date,day_of_week,PX_HIGH,PX_LOW,PX_OPEN,PX_LAST,PX_VOLUME
3788,2018-12-25,1,1883.0,1883.0,1883.0,1883.0,NaN
3792,2019-01-01,1,1853.0,1853.0,1853.0,1853.0,NaN


In [4]:
df.dropna(inplace=True)

df.set_index('date', drop=True, inplace=True)
df.head()

,day_of_week,PX_HIGH,PX_LOW,PX_OPEN,PX_LAST,PX_VOLUME
date,,,,,,
2004-01-02,4,1623.5,1600.0,1604.0,1619.0,1236.0
2004-01-05,0,1623.0,1608.0,1621.0,1615.0,991.0
2004-01-06,1,1623.0,1603.0,1618.0,1608.0,718.0
2004-01-07,2,1609.0,1600.0,1606.0,1600.0,366.0
2004-01-08,3,1623.0,1594.0,1603.0,1620.0,870.0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5444 entries, 2004-01-02 to 2025-07-18
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   day_of_week  5444 non-null   int64  
 1   PX_HIGH      5444 non-null   float64
 2   PX_LOW       5444 non-null   float64
 3   PX_OPEN      5444 non-null   float64
 4   PX_LAST      5444 non-null   float64
 5   PX_VOLUME    5444 non-null   float64
dtypes: float64(5), int64(1)
memory usage: 297.7 KB


In [6]:
# Add all ta features
# df = add_all_ta_features(df, open="PX_OPEN", high="PX_HIGH", low="PX_LOW", close="PX_LAST", volume="PX_VOLUME")
# df.columns

df['SMA20'] = ta.trend.sma_indicator(df.PX_LAST, window=20, fillna=False)
df['SMA50'] = ta.trend.sma_indicator(df.PX_LAST, window=50, fillna=False)

df['EMA20'] = ta.trend.ema_indicator(df.PX_LAST, window=20, fillna=False)
df['EMA50'] = ta.trend.ema_indicator(df.PX_LAST, window=50, fillna=False)

df['ADX'] = ta.trend.adx(df['PX_HIGH'], df['PX_LOW'], df['PX_LAST'], window=14, fillna=False)

df['MACD'] = ta.trend.macd(df['PX_HIGH'], window_slow=26, window_fast=12, fillna=False)
df['MACD_SIG'] = ta.trend.macd_signal(df['PX_HIGH'], window_slow=26, window_fast=12, window_sign=9, fillna=False)

df['RSI'] = ta.momentum.rsi(df['PX_HIGH'], window=14, fillna=False)

df['upper'] = ta.volatility.bollinger_hband(df['PX_HIGH'], window=20, window_dev=2, fillna=False)
df['mid'] = ta.volatility.bollinger_mavg(df['PX_HIGH'], window=20, fillna=False)
df['lower'] = ta.volatility.bollinger_lband(df['PX_HIGH'], window=20, window_dev=2, fillna=False)

In [7]:
df.dropna(inplace=True)
df['date'] = df.index
df.head()

,day_of_week,PX_HIGH,PX_LOW,PX_OPEN,PX_LAST,PX_VOLUME,SMA20,SMA50,EMA20,EMA50,ADX,MACD,MACD_SIG,RSI,upper,mid,lower,date
date,,,,,,,,,,,,,,,,,,
2004-03-11,3,1690.0,1660.0,1667.0,1689.0,1515.0,1707.625,1666.95,1688.307137,1673.987500,35.887244,-2.583332,7.677975,47.792100,1785.940857,1720.425,1654.909143,2004-03-11
2004-03-12,4,1691.0,1663.0,1691.0,1664.5,1152.0,1704.850,1667.86,1686.039790,1673.615441,33.664713,-2.777353,5.586910,48.080923,1785.431428,1718.725,1652.018572,2004-03-12
2004-03-15,0,1676.0,1662.0,1671.0,1673.0,636.0,1702.175,1669.02,1684.797905,1673.591306,31.553195,-4.094295,3.650669,44.136615,1784.832383,1715.925,1647.017617,2004-03-15
2004-03-16,1,1695.0,1675.0,1683.0,1692.0,1029.0,1699.575,1670.70,1685.483819,1674.313216,30.383330,-3.563759,2.207783,49.758799,1781.555225,1713.425,1645.294775,2004-03-16
2004-03-17,2,1697.0,1665.0,1688.0,1668.0,850.0,1696.000,1672.06,1683.818694,1674.065639,28.791736,-2.947939,1.176639,50.325524,1771.786927,1709.525,1647.263073,2004-03-17


In [8]:
plot_length = len(df)
# plot_length = 150
plot_df = df.copy(deep=True).iloc[:plot_length]
plot_df['weekday'] = plot_df['date'].dt.day_name()

fig = px.line(plot_df,
              x="date",
              y="PX_LAST",
              # color="weekday",
              title="{} Price Over Time".format(datasets[dataset_idx]))
fig.show()

### 2. Data processing for modeling

In [9]:
def create_sliding_window(data, feature_length, output_length, stride=1):
    X_list, y_list = [], []
    for i in range(len(data)):
      if (i + feature_length + output_length) < len(data):
        X_list.append(data.iloc[i:i+feature_length:stride, 1:].values)
        y_list.append(data.iloc[i+feature_length:i+feature_length+output_length:stride, 0])
    return np.array(X_list), np.array(y_list)

In [10]:
train_split = 0.7
n_train = int(train_split * len(df))
n_test = len(df) - n_train

feature_array = df.drop(columns=['day_of_week', 'PX_HIGH', 'PX_LOW', 'PX_OPEN', 'PX_VOLUME', 'date']).values
target_array = df.PX_LAST.values

# Fit Scaler only on Training features
feature_scaler = MinMaxScaler()
feature_scaler.fit(feature_array[:n_train])
# Fit Scaler only on Training target values
target_scaler = MinMaxScaler()
target_scaler.fit(target_array[:n_train].reshape(-1, 1))

# Transfom on both Training and Test data
scaled_array = pd.DataFrame(feature_scaler.transform(feature_array),
                            columns=df.drop(columns=['day_of_week', 'PX_HIGH', 'PX_LOW', 'PX_OPEN', 'PX_VOLUME', 'date']).columns)

feature_length = 30
output_length = 10
X, y = create_sliding_window(scaled_array, feature_length, output_length)

X_train = X[:n_train]
y_train = y[:n_train]

X_test = X[n_train:]
y_test = y[n_train:]

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(3776, 30, 11) (3776, 10) (1579, 30, 11) (1579, 10)


### 3. LSTM model

In [11]:
class BayesianLSTM(nn.Module):

    def __init__(self, n_features, output_length, batch_size):

        super(BayesianLSTM, self).__init__()

        self.batch_size = batch_size # user-defined

        self.hidden_size_1 = 128 # number of encoder cells (from paper)
        self.hidden_size_2 = 32 # number of decoder cells (from paper)
        self.stacked_layers = 2 # number of (stacked) LSTM layers for each stage
        self.dropout_probability = 0.5 # arbitrary value (the paper suggests that performance is generally stable across all ranges)

        self.lstm1 = nn.LSTM(n_features,
                             self.hidden_size_1,
                             num_layers=self.stacked_layers,
                             batch_first=True)
        self.lstm2 = nn.LSTM(self.hidden_size_1,
                             self.hidden_size_2,
                             num_layers=self.stacked_layers,
                             batch_first=True)

        self.fc = nn.Linear(self.hidden_size_2, output_length)
        self.loss_fn = nn.MSELoss()

    def forward(self, x):
        batch_size, seq_len, _ = x.size()

        hidden = self.init_hidden1(batch_size)
        output, _ = self.lstm1(x, hidden)
        output = F.dropout(output, p=self.dropout_probability, training=True)
        state = self.init_hidden2(batch_size)
        output, state = self.lstm2(output, state)
        output = F.dropout(output, p=self.dropout_probability, training=True)
        # Important when lstm num_layers > 1
        output = output[:, -1, :] # take the last decoder cell's outputs
        y_pred = self.fc(output)
        return y_pred

    def init_hidden1(self, batch_size):
        hidden_state = Variable(torch.zeros(self.stacked_layers, batch_size, self.hidden_size_1))
        cell_state = Variable(torch.zeros(self.stacked_layers, batch_size, self.hidden_size_1))
        return hidden_state, cell_state

    def init_hidden2(self, batch_size):
        hidden_state = Variable(torch.zeros(self.stacked_layers, batch_size, self.hidden_size_2))
        cell_state = Variable(torch.zeros(self.stacked_layers, batch_size, self.hidden_size_2))
        return hidden_state, cell_state

    def loss(self, pred, truth):
        return self.loss_fn(pred, truth)

    def predict(self, X):
        return self(torch.tensor(X, dtype=torch.float32)).view(-1).detach().numpy()

In [12]:
n_features = scaled_array.shape[-1] - 1

batch_size = 128
n_epochs = 200
learning_rate = 1e-3

bayesian_lstm = BayesianLSTM(n_features=n_features,
                             output_length=output_length,
                             batch_size = batch_size)

criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(bayesian_lstm.parameters(), lr=learning_rate)

In [13]:
bayesian_lstm.train()

for e in range(1, n_epochs+1):
    for b in range(0, len(X_train), batch_size):
        features = X_train[b:b+batch_size,:,:]
        target = y_train[b:b+batch_size].flatten()

        X_batch = torch.tensor(features,dtype=torch.float32)
        y_batch = torch.tensor(target,dtype=torch.float32)

        output = bayesian_lstm(X_batch)
        loss = criterion(output.view(-1), y_batch)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    if e % 10 == 0:
      print('epoch', e, 'loss: ', loss.item())

epoch 10 loss:  0.004277965519577265
epoch 20 loss:  0.0019582558888942003
epoch 30 loss:  0.0011376111069694161
epoch 40 loss:  0.0011362405493855476
epoch 50 loss:  0.0009438177803531289
epoch 60 loss:  0.001137839863076806
epoch 70 loss:  0.0008624814217910171
epoch 80 loss:  0.0009247410926036537
epoch 90 loss:  0.0004696151299867779
epoch 100 loss:  0.0007684663287363946
epoch 110 loss:  0.0005354161839932203
epoch 120 loss:  0.000553412304725498
epoch 130 loss:  0.00048451745533384383
epoch 140 loss:  0.000516432395670563
epoch 150 loss:  0.0010227393358945847
epoch 160 loss:  0.0006863424787297845
epoch 170 loss:  0.0009636234608478844
epoch 180 loss:  0.0009592299466021359
epoch 190 loss:  0.0006765298894606531
epoch 200 loss:  0.0032008022535592318


### 4. Model prediction

In [14]:
def inverse_transform(y):
    return target_scaler.inverse_transform(y.reshape(-1, 1))

In [15]:
offset = feature_length + output_length

training_df = pd.DataFrame()
training_df['date'] = df['date'].iloc[offset:n_train + offset:1]
training_predictions = bayesian_lstm.predict(X_train)[::10]
training_df['Close'] = inverse_transform(training_predictions)
training_df['source'] = 'Training Prediction'

training_truth_df = pd.DataFrame()
training_truth_df['date'] = training_df['date']
training_truth_df['Close'] = df['PX_LAST'].iloc[offset:n_train + offset:1]
training_truth_df['source'] = 'True Values'

testing_df = pd.DataFrame()
testing_df['date'] = df['date'].iloc[n_train + offset::1]
testing_predictions = bayesian_lstm.predict(X_test)[::10]
testing_df['Close'] = inverse_transform(testing_predictions)
testing_df['source'] = 'Test Prediction'

testing_truth_df = pd.DataFrame()
testing_truth_df['date'] = testing_df['date']
testing_truth_df['Close'] = df['PX_LAST'].iloc[n_train + offset::1]
testing_truth_df['source'] = 'True Values'

evaluation = pd.concat([training_df,
                        testing_df,
                        training_truth_df,
                        testing_truth_df
                        ], axis=0)

In [16]:
fig = px.line(evaluation,
                 x="date",
                 y="Close",
                 color="source",
                 title="{} Price Over Time".format(datasets[dataset_idx]))
fig.show()

In [17]:
n_experiments = 100

test_uncertainty_df = pd.DataFrame()
test_uncertainty_df['date'] = testing_df['date']

for i in range(n_experiments):
  experiment_predictions = bayesian_lstm.predict(X_test)[::10]
  test_uncertainty_df['Close_{}'.format(i)] = inverse_transform(experiment_predictions)

Close_df = test_uncertainty_df.filter(like='Close', axis=1)
test_uncertainty_df['Close_mean'] = Close_df.mean(axis=1)
test_uncertainty_df['Close_std'] = Close_df.std(axis=1)

test_uncertainty_df = test_uncertainty_df[['date', 'Close_mean', 'Close_std']]

C:\Users\Di.Sha\AppData\Local\Temp\ipykernel_7256\648296752.py:8: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\Di.Sha\AppData\Local\Temp\ipykernel_7256\648296752.py:11: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\Di.Sha\AppData\Local\Temp\ipykernel_7256\648296752.py:12: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented f

In [18]:
test_uncertainty_df['lower_bound'] = test_uncertainty_df['Close_mean'] - 3*test_uncertainty_df['Close_std']
test_uncertainty_df['upper_bound'] = test_uncertainty_df['Close_mean'] + 3*test_uncertainty_df['Close_std']

In [19]:
test_uncertainty_plot_df = test_uncertainty_df.copy(deep=True)
test_uncertainty_plot_df = test_uncertainty_plot_df
truth_uncertainty_plot_df = testing_truth_df.copy(deep=True)
truth_uncertainty_plot_df = truth_uncertainty_plot_df

upper_trace = go.Scatter(
    x=test_uncertainty_plot_df['date'],
    y=test_uncertainty_plot_df['upper_bound'],
    mode='lines',
    fill=None,
    name='99% Upper Confidence Bound'
    )
lower_trace = go.Scatter(
    x=test_uncertainty_plot_df['date'],
    y=test_uncertainty_plot_df['lower_bound'],
    mode='lines',
    fill='tonexty',
    fillcolor='rgba(255, 211, 0, 0.1)',
    name='99% Lower Confidence Bound'
    )
real_trace = go.Scatter(
    x=truth_uncertainty_plot_df['date'],
    y=truth_uncertainty_plot_df['Close'],
    mode='lines',
    fill=None,
    name='Real Values'
    )

data = [upper_trace, lower_trace, real_trace]

fig = go.Figure(data=data)
fig.update_layout(title='Uncertainty Quantification for {} Price Test Data'.format(datasets[dataset_idx]),
                   xaxis_title='Year',
                   yaxis_title='Price (USD)')

fig.show()

In [20]:
bounds_df = pd.DataFrame()

# Using 99% confidence bounds
bounds_df['lower_bound'] = test_uncertainty_plot_df['lower_bound']
bounds_df['prediction'] = test_uncertainty_plot_df['Close_mean']
bounds_df['real_value'] = truth_uncertainty_plot_df['Close']
bounds_df['upper_bound'] = test_uncertainty_plot_df['upper_bound']

bounds_df['contained'] = ((bounds_df['real_value'] >= bounds_df['lower_bound']) &
                          (bounds_df['real_value'] <= bounds_df['upper_bound']))

print("Proportion of points contained within 99% confidence interval:",
      bounds_df['contained'].mean())

Proportion of points contained within 99% confidence interval: 0.6276124129195694
